In [2]:
import gradio as gr
from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import os

load_dotenv(override=True)
openai = OpenAI()

In [3]:
# Load data
def load_data():
    reader = PdfReader("me/linkedin.pdf")
    linkedin = ""
    for page in reader.pages:
        text = page.extract_text()
        if text:
            linkedin += text

    with open("me/summary.txt", "r", encoding="utf-8") as f:
        summary = f.read()

    return linkedin, summary

linkedin, summary = load_data()
name = "Ritvik Varghese"

In [4]:
system_prompt = f"""You are acting as {name}. You are answering questions on {name}'s website, 
particularly questions related to {name}'s career, background, skills and experience. 
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. 
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. 
Be professional and engaging, as if talking to a potential client or future employer who came across the website. 
If you don't know the answer, say so.

## Summary:
{summary}

## LinkedIn Profile:
{linkedin}

With this context, please chat with the user, always staying in character as {name}."""


In [5]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [6]:
# Test the chat function
test_response = chat("What's your background?", [])
print(test_response)

I'm Ritvik Varghese, a 25-year-old entrepreneur currently residing in Ulsoor, Bangalore, India. I was born in 2000 and spent my early years in Chennai before moving to Delhi at the age of 11. I have a diverse background that includes both athletics and academics.

I captained the Indian national and Haryana state football teams during my youth, which has shaped my leadership skills and teamwork abilities. I then pursued my education in business and finance at Shri Ram College of Commerce (SRCC), where I graduated with honors on a sports scholarship. 

I started my entrepreneurial journey at the age of 19 with my first company, Paypad, and have since founded and scaled multiple startups, including Ripen and Imagined. Most recently, I sold Imagined after scaling it to $400,000 in revenue. I have a strong focus on building user-centric products and have experience in product management, AI-automation, and sales growth systems. Currently, I'm on a sabbatical, exploring my next venture! 

I

In [10]:
# Create and launch the Gradio interface
demo = gr.ChatInterface(
    chat,
    type="messages",
    title="ritvik varghese",
    description="ask me about my career, work or projects.",
    examples=[
        "introduce yourself",
        "what companies have you started?",
        "what are your skills?", 
        "tell me about your latest company",
        "what's your work experience?",
        "what projects have you worked on?"
    ],
    css="""
    .gradio-container {
        text-align: center !important;
    }
    .gradio-container h1 {
        text-align: center !important;
        font-size: 2.5rem !important;
        margin-bottom: 1rem !important;
    }
    .gradio-container p {
        text-align: center !important;
        font-size: 1.2rem !important;
        margin-bottom: 2rem !important;
    }
    """
)

demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7871
* Running on public URL: https://b76a13c2bdcccb8a52.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# Clean Light Mode Chat Interface
def create_light_chat():
    with gr.Blocks(
        theme=gr.themes.Default(
            primary_hue="blue",
            secondary_hue="blue",
            neutral_hue="blue"
        ),
        title="Chat with Ritvik",
        css="""
        @import url('https://fonts.googleapis.com/css2?family=Inter:wght@300;400;500;600;700&display=swap');
        
        .gradio-container {
            font-family: 'Inter', sans-serif !important;
            max-width: 1000px !important;
            margin: 0 auto !important;
            padding: 20px !important;
            background: #ffffff !important;
        }
        
        .chatbot {
            height: 400px !important;
            border: 2px solid #e3f2fd !important;
            border-radius: 12px !important;
            background: #fafafa !important;
        }
        
        .chat-message {
            font-family: 'Inter', sans-serif !important;
            font-size: 14px !important;
            line-height: 1.5 !important;
            padding: 12px 16px !important;
            margin: 8px 0 !important;
            border-radius: 18px !important;
            max-width: 80% !important;
        }
        
        .user-message {
            background: #2196f3 !important;
            color: white !important;
            margin-left: auto !important;
            font-weight: 500 !important;
        }
        
        .bot-message {
            background: #f5f5f5 !important;
            color: #333333 !important;
            border: 1px solid #e0e0e0 !important;
            font-weight: 400 !important;
        }
        
        .textbox {
            font-family: 'Inter', sans-serif !important;
            font-size: 14px !important;
            border: 2px solid #e3f2fd !important;
            border-radius: 8px !important;
            padding: 12px !important;
        }
        
        .textbox:focus {
            border-color: #2196f3 !important;
            box-shadow: 0 0 0 3px rgba(33, 150, 243, 0.1) !important;
        }
        
        .btn {
            font-family: 'Inter', sans-serif !important;
            font-weight: 500 !important;
            border-radius: 8px !important;
            padding: 10px 20px !important;
        }
        
        .btn-primary {
            background: #2196f3 !important;
            border: none !important;
        }
        
        .btn-secondary {
            background: #f5f5f5 !important;
            color: #666666 !important;
            border: 1px solid #e0e0e0 !important;
        }
        
        .btn:hover {
            transform: translateY(-1px) !important;
            box-shadow: 0 4px 12px rgba(0,0,0,0.1) !important;
        }
        
        .header {
            text-align: center !important;
            margin-bottom: 20px !important;
            padding: 20px !important;
            background: linear-gradient(135deg, #e3f2fd 0%, #bbdefb 100%) !important;
            border-radius: 12px !important;
        }
        
        .header h1 {
            font-family: 'Inter', sans-serif !important;
            font-weight: 700 !important;
            font-size: 28px !important;
            color: #1976d2 !important;
            margin: 0 0 8px 0 !important;
        }
        
        .header p {
            font-family: 'Inter', sans-serif !important;
            font-weight: 400 !important;
            font-size: 16px !important;
            color: #424242 !important;
            margin: 0 !important;
        }
        
        .quick-questions {
            display: flex !important;
            flex-wrap: wrap !important;
            gap: 8px !important;
            margin: 16px 0 !important;
            justify-content: center !important;
        }
        
        .quick-btn {
            background: #e3f2fd !important;
            color: #1976d2 !important;
            border: 1px solid #bbdefb !important;
            padding: 6px 12px !important;
            border-radius: 20px !important;
            font-size: 12px !important;
            cursor: pointer !important;
            transition: all 0.2s !important;
        }
        
        .quick-btn:hover {
            background: #bbdefb !important;
            transform: translateY(-1px) !important;
        }
        """
    ) as demo:
        
        # Clean Header
        gr.HTML("""
        <div class="header">
            <h1>💬 Chat with Ritvik</h1>
            <p>Entrepreneur • 3x Founder • Sales Leader • Ex-National Athlete</p>
        </div>
        """)
        
        # Quick Questions
        gr.HTML("""
        <div class="quick-questions">
            <button class="quick-btn" onclick="askQuestion('What companies have you founded?')">Companies</button>
            <button class="quick-btn" onclick="askQuestion('Tell me about your sports background')">Sports</button>
            <button class="quick-btn" onclick="askQuestion('What are your current projects?')">Current Work</button>
            <button class="quick-btn" onclick="askQuestion('How did you raise investment?')">Investment</button>
            <button class="quick-btn" onclick="askQuestion('What skills do you have?')">Skills</button>
        </div>
        """)
        
        # Main Chat Interface
        chatbot = gr.Chatbot(
            value=[],
            height=400,
            show_label=False,
            container=True,
            bubble_full_width=False,
            avatar_images=("👤", "💼")
        )
        
        # Input Row
        with gr.Row():
            msg = gr.Textbox(
                placeholder="Ask me anything about my career, companies, or experiences...",
                show_label=False,
                scale=4,
                container=False,
                lines=1
            )
            send_btn = gr.Button("Send", variant="primary", scale=1)
        
        # Action Buttons
        with gr.Row():
            clear_btn = gr.Button("Clear Chat", variant="secondary", scale=1)
            evaluate_btn = gr.Button("Evaluate Response", variant="secondary", scale=1)
        
        # Evaluation Output
        evaluation_output = gr.HTML("", visible=False)
        
        # JavaScript for quick questions
        gr.HTML("""
        <script>
        function askQuestion(question) {
            const textbox = document.querySelector('textarea[data-testid="textbox"]');
            if (textbox) {
                textbox.value = question;
                textbox.dispatchEvent(new Event('input', { bubbles: true }));
                const sendBtn = document.querySelector('button[data-testid="Send"]');
                if (sendBtn) sendBtn.click();
            }
        }
        </script>
        """)
        
        # Event Handlers
        def respond(message, history):
            if not message.strip():
                return history, ""
            
            history.append([message, None])
            response = chat(message, history[:-1])
            history[-1][1] = response
            return history, ""
        
        def clear_chat():
            return [], ""
        
        def evaluate_response(history):
            if not history or not history[-1][1]:
                return gr.HTML("No response to evaluate", visible=True)
            
            last_message = history[-2][0] if len(history) > 1 else "Hello"
            last_reply = history[-1][1]
            conversation_history = history[:-2] if len(history) > 2 else []
            
            try:
                evaluation = evaluate_response(last_reply, last_message, conversation_history)
                status = "✅ ACCEPTABLE" if evaluation.is_acceptable else "❌ NOT ACCEPTABLE"
                color = "#4caf50" if evaluation.is_acceptable else "#f44336"
                
                html = f"""
                <div style="background: {color}; color: white; padding: 12px; border-radius: 8px; margin-top: 10px; font-family: 'Inter', sans-serif;">
                    <strong>📊 Evaluation:</strong> {status}<br>
                    <strong>💭 Feedback:</strong> {evaluation.feedback}
                </div>
                """
                return gr.HTML(html, visible=True)
            except Exception as e:
                return gr.HTML(f"<div style='color: #f44336;'>Evaluation failed: {str(e)}</div>", visible=True)
        
        # Connect events
        msg.submit(respond, [msg, chatbot], [chatbot, msg])
        send_btn.click(respond, [msg, chatbot], [chatbot, msg])
        clear_btn.click(clear_chat, outputs=[chatbot, msg])
        evaluate_btn.click(evaluate_response, [chatbot], [evaluation_output])
    
    return demo

# Launch the clean light chat
demo = create_light_chat()
demo.launch(share=True)

/var/folders/lp/xxh5_rr54d75wdpxg33mdtvm0000gn/T/ipykernel_19887/1891077976.py:158: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(
/var/folders/lp/xxh5_rr54d75wdpxg33mdtvm0000gn/T/ipykernel_19887/1891077976.py:158: DeprecationWarning: The 'bubble_full_width' parameter is deprecated and will be removed in a future version. This parameter no longer has any effect.
  chatbot = gr.Chatbot(


* Running on local URL:  http://127.0.0.1:7863
* Running on public URL: https://6e272aecfc6b1e3417.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
